In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

In [ ]:
all_csvs = glob.glob('./out/v3/ls*_aea_merged.csv')
all_csvs.sort()

In [ ]:
def read_process_csv_to_gdf(csv):
    temp_df = pd.read_csv(csv)
    temp_df['satellite'] = os.path.basename(csv)[:3]
    temp_df['year'] = int(os.path.basename(csv)[4:8])
    temp_df = temp_df.loc[temp_df['hydropoly_max']<100]
    temp_df['area_ha'] = temp_df['area']*100/10000 # HA
    temp_df['area_km'] = temp_df['area']*100/(1000*1000) # km2
    temp_df = temp_df.loc[temp_df['area_ha']<100] # Remove greater than 100 ha
    temp_gdf = gpd.GeoDataFrame(
        temp_df, geometry=gpd.points_from_xy(temp_df.center_lon, temp_df.center_lat),
        crs='ESRI:102033'
    )
    return temp_gdf

In [ ]:
def sjoin_summarize(points_gdf, poly_gdf, poly_field):
    
    joined_gdf = gpd.sjoin(points_gdf, poly_gdf, predicate='within', how='inner')
    return joined_gdf[['area_ha', poly_field]].groupby(poly_field).agg(['sum', 'count', 'median'])['area_ha']

In [ ]:
muni_gdf = gpd.read_file('../clip/brazil_munis_interviews_aea.shp')
muni_gdf['CD_MUN'] = muni_gdf['CD_MUN'].astype(int)

In [ ]:
for in_csv in all_csvs:
    muni_out_csv = './out/v3_interviews_only/{}'.format(os.path.basename(in_csv).replace('merged.csv', 'muni_stats.csv'))
    if not os.path.isfile(muni_out_csv):
        res_gdf = read_process_csv_to_gdf(in_csv)
        muni_results = sjoin_summarize(res_gdf, muni_gdf, 'CD_MUN')
        muni_results.to_csv(muni_out_csv)
    print(os.path.basename(in_csv), "Done")

# Make figures

In [ ]:
def read_process_region_csv(csv):
    temp_df = pd.read_csv(csv)
    temp_df['satellite'] = os.path.basename(csv)[:3]
    temp_df['year'] = int(os.path.basename(csv)[4:8])

    return temp_df

muni_csvs = glob.glob('./out/v3_interviews_only/*.csv')
muni_csvs.sort()
muni_list = [read_process_region_csv(csv) for csv in muni_csvs]
muni_df = pd.concat(muni_list).set_index('CD_MUN')
# muni_df = muni_df.sort_index()

In [ ]:
muni_df = muni_df.loc[~((muni_df['satellite']=='ls7') & (muni_df['year']>2019))]
# muni_df = muni_df.loc[~((muni_df['satellite']=='ls8') & (muni_df['year']<2025))]
muni_df = muni_df.loc[~((muni_df['satellite']=='ls7') & (muni_df['year']==2000))]
muni_df = muni_df.loc[~((muni_df['satellite']=='ls5') & (muni_df['year']==2001))]
muni_df = muni_df.loc[~((muni_df['satellite']=='ls5') & (muni_df['year']==2002))]

In [ ]:
muni_names_dict = {
    5100201: 'Água Boa',
    5101803: 'Barra do Garças',
    5102702: 'Canarana',
    5106257: 'Nova Xavantina',
    5107065: 'Querência',
}
muni_order = ['Barra do Garças', 'Nova Xavantina', 'Canarana', 'Querência']
muni_colors = ["#fd7f6f", "#7eb0d5", "#b2e061", "#bd7ebe"]
muni_styles = ['-','--','-.',':']

In [ ]:
muni_df['AREA_KM2'] = muni_gdf.set_index('CD_MUN')['AREA_KM2']
muni_df['density'] = muni_df['count']/muni_df['AREA_KM2']
muni_df = muni_df.drop(columns='AREA_KM2').reset_index()

In [ ]:
muni_count_df = muni_df.drop(
    columns=['satellite']
    ).groupby(
        ['year', 'CD_MUN']
        ).mean().unstack(
        # ).mean()[['count','sum', 'median', 'density']].unstack(
            level=1
            ).rename(columns=muni_names_dict, level=1).drop(
                columns='Água Boa', level=1
            )
muni_count_df = muni_count_df.swaplevel(i=0, j=1, axis=1)[muni_order].swaplevel(i=0,j=1,axis=1)[['sum', 'count', 'median', 'density']]

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(7.35, 3))
muni_count_df['count'].plot.area(ax=axs[0], legend=False, color=muni_colors, lw=0.1)
muni_count_df['density'].plot(ax=axs[1], legend=False, color=muni_colors, lw=1.8, style=muni_styles)
handles, labels = axs[1].get_legend_handles_labels()
axs[0].legend(handles[::-1], labels[::-1])
axs[0].set_ylabel('Reservoir Count')
axs[1].set_ylabel('Reservoir Density (# per km$^2$)')
for ax in axs:
    ax.set_xlabel('Year')
    ax.set_xlim([1984, 2023])

fig.tight_layout()
# muni_count_df['sum'].plot()

In [ ]:
grayscale_colors = plt.cm.gray([0.8, 0.4, 0.6, 0.2])
grayscale_styles = [':', '--', '-.','-']

In [ ]:
# Portuguese
fig, axs = plt.subplots(1,2, figsize=(7.35, 3))
muni_count_df['count'].drop(index=2014).plot.area(ax=axs[0], legend=False, color=grayscale_colors, lw=0.1)
muni_count_df['density'].drop(index=2014).plot(ax=axs[1], legend=False, color=grayscale_colors, lw=1.8, style=grayscale_styles)
handles, labels = axs[1].get_legend_handles_labels()
axs[0].legend(handles[::-1], labels[::-1])
axs[0].set_ylabel('Número das represas')
axs[1].set_ylabel('Densidade das represas\n(número por km$^2$)')
for ax in axs:
    ax.set_xlabel('Ano')
    ax.set_xlim([1984, 2023])

fig.tight_layout()
# muni_count_df['sum'].plot()